In [ ]:
import numpy
import scipy.integrate
import pyfabm

%matplotlib widget
from matplotlib import pyplot

In [ ]:
# Create model (loads fabm.yaml)
model = pyfabm.Model('../extern/fabm/testcases/fabm-gotm-npzd.yaml')

# Configure the environment
# Note: the set of environmental dependencies depends on the loaded biogeochemical model.
model.cell_thickness = 1.
model.findDependency('surface_downwelling_photosynthetic_radiative_flux').value = 50.
model.findDependency('downwelling_photosynthetic_radiative_flux').value = 25.

# Verify the model is ready to be used
assert model.checkReady(), 'One or more model dependencies have not been fulfilled.'

In [ ]:
# Time derivative
def dy(y, t0):
    model.state[:] = y
    return model.getRates()

# Time-integrate over 200 days (note: FABM's internal time unit is seconds!)
t = numpy.linspace(0, 200., 100)
y = scipy.integrate.odeint(dy, model.state, t*86400)

In [ ]:
# Plot results
import pylab
pylab.plot(t, y)
pylab.legend([variable.path for variable in model.state_variables])
pylab.grid()